In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff

save_directory = r'C:\Users\Jaber\OneDrive - University of Florida\Educational\Chapters\Chapter1_Abstain\Codes\Results\Results_Transformer_Autoencoder'

# Define the range of lambda values
lambdas = np.arange(0.5, 0.95, 0.05)

# Initialize dictionaries to store metrics for each lambda
metrics_dict = {l: {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'specificity': []} for l in lambdas}

# Function to classify based on probability and threshold
def classify_with_reject(probabilities, threshold, initial_fp_fn_indices):
    predictions = []
    abstain_instances = []  # To store indices of abstain instances
    for i, prob in enumerate(probabilities):
        if i not in initial_fp_fn_indices:
            if prob >= 0.5:
                predictions.append(1)
            else:
                predictions.append(0)
        else:
            if prob >= threshold:
                predictions.append(1)  # Classify as positive if prob is >= threshold
            elif prob < 1 - threshold:
                predictions.append(0)  # Classify as negative if prob is < 1 - threshold
            else:
                predictions.append(-1)  # Reject classification for uncertain predictions
                abstain_instances.append(i)  # Record abstain instance index
    return np.array(predictions), abstain_instances


# Load new dataset
new_data_path = r"C:\Users\Jaber\OneDrive - University of Florida\Educational\Chapters\Chapter1_Abstain\Datasets\ShandsData\combined_BPM_data.csv"

# Load the new data
new_df = pd.read_csv(new_data_path)

# Store instance names
instance_names = new_df.iloc[:, 0].values  # Save the first column as instance names

# Remove the first column which contains non-numeric data (names of instances)
new_df = new_df.iloc[:, 1:]

# Convert labels to binary (1 or 2 to 0 or 1)
new_df['label'] = new_df['label'].apply(lambda x: 0 if x == 1 else 1)

# Treat missing values with mean strategy only for numeric columns
numeric_columns = new_df.select_dtypes(include=[np.number]).columns
new_df[numeric_columns] = new_df[numeric_columns].fillna(new_df[numeric_columns].mean())

# Extract features (FHR time series) and labels
X_new_time_series = new_df.iloc[:, :-1].values  # Exclude the label column
y_new = new_df['label'].values  # Include the target variable y

# Autoencoder for Dimensionality Reduction
input_dim = X_new_time_series.shape[1]
encoding_dim = 30

input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(128, activation="relu")(input_layer)
encoder = tf.keras.layers.Dense(64, activation="relu")(encoder)
encoder = tf.keras.layers.Dense(encoding_dim, activation="relu")(encoder)
decoder = tf.keras.layers.Dense(64, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(128, activation='relu')(decoder)
decoder = tf.keras.layers.Dense(input_dim, activation='sigmoid')(decoder)

autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the Autoencoder
autoencoder.fit(X_new_time_series, X_new_time_series, epochs=50, batch_size=32, shuffle=True, verbose=0)

# Encode the data
encoder_model = tf.keras.models.Model(inputs=input_layer, outputs=encoder)
X_new_time_series_reduced = encoder_model.predict(X_new_time_series)

# Leave-One-Out Cross Validation
loo = LeaveOneOut()

fold_number = 1
for train_index, test_index in loo.split(X_new_time_series_reduced):
    X_train_new, X_test_new = X_new_time_series_reduced[train_index], X_new_time_series_reduced[test_index]
    y_train_new, y_test_new = y_new[train_index], y_new[test_index]

    # Standardize the features (mean=0, std=1)
    scaler_time_series_new = StandardScaler()
    X_train_new_time_series = scaler_time_series_new.fit_transform(X_train_new)
    X_test_new_time_series = scaler_time_series_new.transform(X_test_new)

    # Define the Transformer model
    def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
        # Normalization and Attention
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = tf.keras.layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout
        )(x, x)
        x = tf.keras.layers.Dropout(dropout)(x)
        res = x + inputs

        # Feed Forward Part
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)
        x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
        x = tf.keras.layers.Dropout(dropout)(x)
        x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        return x + res

    def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
        inputs = tf.keras.Input(shape=input_shape)
        x = inputs
        for _ in range(num_transformer_blocks):
            x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

        x = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
        for dim in mlp_units:
            x = tf.keras.layers.Dense(dim, activation="relu")(x)
            x = tf.keras.layers.Dropout(mlp_dropout)(x)
        outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
        return tf.keras.Model(inputs, outputs)

    input_shape = (X_train_new_time_series.shape[1], 1)
    transformer_model = build_model(
        input_shape,
        head_size=256,
        num_heads=4,
        ff_dim=4,
        num_transformer_blocks=4,
        mlp_units=[128],
        dropout=0.25,
        mlp_dropout=0.4,
    )

    transformer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with early stopping
    best_model_new = None
    best_score_new = 0
    no_improvement_epochs_new = 0
    patience_new = 3  # Patience for early stopping

    for epoch in range(20):  # Number of epochs
        history_new = transformer_model.fit(X_train_new_time_series, y_train_new, epochs=1, batch_size=8, validation_split=0.1, verbose=0)

        # Evaluate on the training data
        train_accuracy_new = history_new.history['accuracy'][-1]
        
        # Check for early stopping
        if train_accuracy_new > best_score_new:
            best_model_new = transformer_model
            best_score_new = train_accuracy_new
            no_improvement_epochs_new = 0
        else:
            no_improvement_epochs_new += 1
        
        if no_improvement_epochs_new >= patience_new:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    # Use the best model for predictions
    test_probabilities_new = best_model_new.predict(X_test_new_time_series).flatten()

    # Initial classification with lambda=0.5
    initial_predictions_new = (test_probabilities_new >= 0.5).astype(int)
    cm_initial_new = confusion_matrix(y_test_new, initial_predictions_new, labels=[0, 1])
    tn_initial_new, fp_initial_new, fn_initial_new, tp_initial_new = cm_initial_new.ravel()

    initial_fp_fn_indices_new = [i for i in range(len(initial_predictions_new)) if initial_predictions_new[i] != y_test_new[i]]

    # Initialize lists to store confusion matrix elements
    tp_list_new = []
    tn_list_new = []
    fp_list_new = []
    fn_list_new = []

    # Initialize a table to store results for each lambda
    table_data_new = []
    abstain_table_data_new = []
    metrics_table_data_new = []

    # Name of the tested instance in this fold
    tested_instance_name = instance_names[test_index[0]]
    print(f"Tested Instance in Fold {fold_number}: {tested_instance_name}")

    # Loop through lambda values and calculate metrics
    for reject_threshold_new in lambdas:
        predictions_new, abstain_indices_new = classify_with_reject(test_probabilities_new, reject_threshold_new, initial_fp_fn_indices_new)
        
        # Filter out abstained instances
        filtered_indices_new = [i for i in range(len(predictions_new)) if predictions_new[i] != -1]
        y_test_filtered_new = y_test_new[filtered_indices_new]
        predictions_filtered_new = predictions_new[filtered_indices_new]
        
        # Calculate confusion matrix elements
        if len(predictions_filtered_new) > 0:
            cm_new = confusion_matrix(y_test_filtered_new, predictions_filtered_new, labels=[0, 1])
            tn_new, fp_new, fn_new, tp_new = cm_new.ravel()
        else:
            cm_new = np.array([[0, 0], [0, 0]])
            tn_new, fp_new, fn_new, tp_new = 0, 0, 0, 0

        # Append confusion matrix elements to lists
        tp_list_new.append(tp_new)
        tn_list_new.append(tn_new)
        fp_list_new.append(fp_new)
        fn_list_new.append(fn_new)
        
        # Append data to table
        table_data_new.append([round(reject_threshold_new, 2), tn_new, fp_new, fn_new, tp_new])
        
        # Report abstain instances (rows of data)
        abstain_instances_info_new = []
        for idx_new in abstain_indices_new:
            abstain_instances_info_new.append((idx_new, y_test_new[idx_new]))
        
        abstain_table_data_new.append([round(reject_threshold_new, 2), abstain_instances_info_new])

        # Calculate and store metrics
        if len(y_test_filtered_new) > 0:
            accuracy_new = accuracy_score(y_test_filtered_new, predictions_filtered_new) * 100
            precision_new = precision_score(y_test_filtered_new, predictions_filtered_new, zero_division=0) * 100
            recall_new = recall_score(y_test_filtered_new, predictions_filtered_new, zero_division=0) * 100
            f1_new = f1_score(y_test_filtered_new, predictions_filtered_new, zero_division=0) * 100
            specificity_new = (tn_new / (tn_new + fp_new)) * 100 if (tn_new + fp_new) > 0 else 0
        else:
            accuracy_new = precision_new = recall_new = f1_new = specificity_new = 0

        metrics_dict[reject_threshold_new]['accuracy'].append(accuracy_new)
        metrics_dict[reject_threshold_new]['precision'].append(precision_new)
        metrics_dict[reject_threshold_new]['recall'].append(recall_new)
        metrics_dict[reject_threshold_new]['f1'].append(f1_new)
        metrics_dict[reject_threshold_new]['specificity'].append(specificity_new)
        
        metrics_table_data_new.append([round(reject_threshold_new, 2), f"{accuracy_new:.2f}%", f"{precision_new:.2f}%", f"{recall_new:.2f}%", f"{f1_new:.2f}%", f"{specificity_new:.2f}%"])

        # Plot confusion matrix for this lambda
        if cm_new.shape != (2, 2):
            cm_padded_new = np.zeros((2, 2), dtype=int)
            cm_padded_new[:cm_new.shape[0], :cm_new.shape[1]] = cm_new
        else:
            cm_padded_new = cm_new

        x_labels_new = ['Normal', 'Abnormal']
        y_labels_new = ['Abnormal', 'Normal']  # Reverse the order of y_labels
        cm_reversed_new = cm_padded_new[::-1]
        fig_new = ff.create_annotated_heatmap(z=cm_reversed_new, x=x_labels_new, y=y_labels_new, colorscale='Blues')
        fig_new.update_layout(
            title=f'Confusion Matrix, Fold {fold_number}, Lambda {reject_threshold_new:.2f}',
            xaxis=dict(title='Predicted labels', tickfont=dict(size=10)),  # Adjust tick font size
            yaxis=dict(title='True labels', tickfont=dict(size=10)),  # Adjust tick font size
            width=400,  # Adjust width
            height=300,  # Adjust height
            margin=dict(l=50, r=50, t=130, b=50)  # Corrected margin specification
        )
        fig_new.show()

    # Plot confusion matrix elements vs. lambda
    plt.figure(figsize=(8, 5))
    plt.plot(lambdas, tp_list_new, marker='o', linestyle='-', label='True Positives (TP)')
    plt.plot(lambdas, tn_list_new, marker='o', linestyle='-', label='True Negatives (TN)')
    plt.plot(lambdas, fp_list_new, marker='o', linestyle='-', label='False Positives (FP)')
    plt.plot(lambdas, fn_list_new, marker='o', linestyle='-', label='False Negatives (FN)')
    plt.xlabel('Lambda (Abstain Threshold)')
    plt.ylabel('Count')
    plt.title(f'Confusion Matrix Elements vs. Lambda Threshold, Fold {fold_number}')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Create a DataFrame for the table and display it
    df_table_cm_new = pd.DataFrame(table_data_new, columns=['Lambda Threshold', 'True Negatives (TN)', 'False Positives (FP)', 'False Negatives (FN)', 'True Positives (TP)'])
    fig_table_cm_new = go.Figure(data=[go.Table(
        header=dict(values=list(df_table_cm_new.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[df_table_cm_new[col].tolist() for col in df_table_cm_new.columns], fill=dict(color=['lavender', 'white']), align='left')
    )])
    fig_table_cm_new.update_layout(width=1000, height=500)  # Adjust the size as needed to fit the table and ensure all entries are visible
    fig_table_cm_new.show()

    # Save the DataFrame to an Excel file
    df_table_cm_new.to_excel(f'{save_directory}/Lambda_Abstain_Confusion_Matrix_Elements_Fold_{fold_number}.xlsx', index=False)
  
    # Create a DataFrame for abstain instances table and display it
    df_abstain_table_new = pd.DataFrame(abstain_table_data_new, columns=['Lambda Threshold', 'Abstain Instances (Index, True Label)'])
    df_abstain_table_new['Tested Instance'] = tested_instance_name
    fig_abstain_table_new = go.Figure(data=[go.Table(
        header=dict(values=list(df_abstain_table_new.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[df_abstain_table_new[col].tolist() for col in df_abstain_table_new.columns], fill=dict(color=['lavender', 'white']), align='left')
    )])
    fig_abstain_table_new.update_layout(width=1000, height=500)  # Adjust the size as needed to fit the table and ensure all entries are visible
    fig_abstain_table_new.show()

    # Save the abstain instances DataFrame to an Excel file
    df_abstain_table_new.to_excel(f'{save_directory}/Lambda_Abstain_Instances_Fold_{fold_number}.xlsx', index=False)
    
    # Create a DataFrame for the performance metrics table and display it
    df_metrics_table_new = pd.DataFrame(metrics_table_data_new, columns=['Lambda Threshold', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Specificity'])
    df_metrics_table_new['Tested Instance'] = tested_instance_name
    fig_metrics_table_new = go.Figure(data=[go.Table(
        header=dict(values=list(df_metrics_table_new.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[df_metrics_table_new[col].tolist() for col in df_metrics_table_new.columns], fill=dict(color=['lavender', 'white']), align='left')
    )])
    fig_metrics_table_new.update_layout(width=1000, height=500)  # Adjust the size as needed to fit the table and ensure all entries are visible
    fig_metrics_table_new.show()

    # Save the performance metrics DataFrame to an Excel file
    df_metrics_table_new.to_excel(f'{save_directory}/Lambda_Abstain_Results_Metrics_Fold_{fold_number}.xlsx', index=False)

    fold_number += 1

# Calculate average metrics for each lambda across all folds
avg_metrics_data_new = []
for l in lambdas:
    avg_accuracy_new = np.mean(metrics_dict[l]['accuracy'])
    avg_precision_new = np.mean(metrics_dict[l]['precision'])
    avg_recall_new = np.mean(metrics_dict[l]['recall'])
    avg_f1_new = np.mean(metrics_dict[l]['f1'])
    avg_specificity_new = np.mean(metrics_dict[l]['specificity'])
    
    avg_metrics_data_new.append([round(l, 2), f"{avg_accuracy_new:.2f}%", f"{avg_precision_new:.2f}%", f"{avg_recall_new:.2f}%", f"{avg_f1_new:.2f}%", f"{avg_specificity_new:.2f}%"])

# Create a DataFrame for average metrics and display it
df_avg_metrics_new = pd.DataFrame(avg_metrics_data_new, columns=['Lambda', 'Average Accuracy', 'Average Precision', 'Average Recall', 'Average F1-score', 'Average Specificity'])
fig_avg_metrics_new = go.Figure(data=[go.Table(
    header=dict(values=list(df_avg_metrics_new.columns), fill_color='paleturquoise', align='left'),
    cells=dict(values=[df_avg_metrics_new[col].tolist() for col in df_avg_metrics_new.columns], fill=dict(color=['lavender', 'white']), align='left')
)])
fig_avg_metrics_new.update_layout(width=1000, height=500)  # Adjust the size as needed to fit the table and ensure all entries are visible
fig_avg_metrics_new.show()

# Save the average metrics DataFrame to an Excel file
df_avg_metrics_new.to_excel(f'{save_directory}/Average_Metrics_Per_Lambda.xlsx', index=False)

# Plot the average performance metrics vs. lambda
plt.figure(figsize=(10, 6))
plt.plot(df_avg_metrics_new['Lambda'], df_avg_metrics_new['Average Accuracy'].str.rstrip('%').astype(float), marker='o', linestyle='-', label='Average Accuracy')
plt.plot(df_avg_metrics_new['Lambda'], df_avg_metrics_new['Average Precision'].str.rstrip('%').astype(float), marker='o', linestyle='-', label='Average Precision')
plt.plot(df_avg_metrics_new['Lambda'], df_avg_metrics_new['Average Recall'].str.rstrip('%').astype(float), marker='o', linestyle='-', label='Average Recall')
plt.plot(df_avg_metrics_new['Lambda'], df_avg_metrics_new['Average F1-score'].str.rstrip('%').astype(float), marker='o', linestyle='-', label='Average F1-score')
plt.plot(df_avg_metrics_new['Lambda'], df_avg_metrics_new['Average Specificity'].str.rstrip('%').astype(float), marker='o', linestyle='-', label='Average Specificity')
plt.xlabel('Lambda (Abstain Threshold)')
plt.ylabel('Percentage')
plt.title('Average Performance Metrics vs. Lambda Threshold')
plt.legend()
plt.grid(True)
plt.show()

print("\nAverage metrics for each lambda have been saved to 'Average_Metrics_Per_Lambda.xlsx'.")
